<a href="https://colab.research.google.com/github/mkane968/Text-Mining-Experiments/blob/main/Topic_Modeling_with_SciFi_Corpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

What are the most common topics in a corpus of science fiction texts? Which texts tend to address the same topics, and why? These are just a couple of the many questions which this code addresses. Below, Gensim is used to create LDA topic models of a pre-loaded dataframe of texts. 

This code is adapted from this [Intro to Topic Modeling with Gensim and pyLDAvis](https://github.com/hawc2/text-analysis-with-python/blob/master/Topic_Modeling.ipynb) and works well with input from from the Text Sectioning and Disaggregation code from [this repository](https://github.com/SF-Nexus/Extracted-Features/blob/main/Text_Sectioning_and_Disaggregation_in_Python.ipynb).

#Install Packages

In [ ]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

In [ ]:
!pip3 install spacy
!python -m spacy download en_core_web_lg

In [ ]:
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.utils import simple_preprocess

In [ ]:
import re

# Retrieve and Convert File to Data Frame

In [ ]:
#Mount Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#Upload files
#Recommended file type: csv file containing "bag of words" like chapters_sample.csv in this repo
from google.colab import files

uploaded = files.upload()

In [ ]:
#Install necessary packages
import numpy as np
import pandas as pd
import io

#Convert csv to dataframe
df = pd.read_csv(io.StringIO(uploaded['word_chunks_bag_of_words_output (1).csv'].decode('utf-8')))
df

In [ ]:
#Add values in Text column to new list (for further cleaning)
data = df.Text.values.tolist()

In [ ]:
#View dataframe as Colab data table
%load_ext google.colab.data_table 
df

# Clean Texts

In [ ]:
#Define list of stopwords
stop_words = stopwords.words('english')

# Add further stopwords by simply "appending" desired words to dictionary
#stop_words.append('movie')

In [ ]:
#Remove punctuation
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
data = [re.sub('\s+', ' ', sent) for sent in data]
data = [re.sub("\'", "", sent) for sent in data]

In [ ]:
#Define function to perform simple preprocessing on text
def sent_to_words(sentences):
    for sentence in sentences:
      yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

In [ ]:
#Run processing function on texts
data_words = list(sent_to_words(data))
print(data_words[:10])

In [ ]:
#Define bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=1, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [ ]:
#Define stopword removal
def remove_stopwords(texts):
   return [[word for word in simple_preprocess(str(doc))
if word not in stop_words] for doc in texts]

#Define function to make bigrams
def make_bigrams(texts):
   return [bigram_mod[doc] for doc in texts]

#def make_trigrams(texts):
#   return [trigram_mod[bigram_mod[doc]] for doc in texts]

#Define function to lemmatize texts
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
   texts_out = []
   for sent in texts:
     doc = nlp(" ".join(sent))
     texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
   return texts_out

In [ ]:
#Run functions to remove stopwords, make bigrams, and lemmatize text
data_words_nostops = remove_stopwords(data_words)
data_words_bigrams = make_bigrams(data_words_nostops)
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=[
   'NOUN', 'ADJ', 'VERB', 'ADV'
])
print(data_lemmatized[:4])

# Building Dictionary and Corpus

In [ ]:
id2word = corpora.Dictionary(data_lemmatized)
texts = data_lemmatized
corpus = [id2word.doc2bow(text) for text in texts]
print(corpus)

# Create Topic Model - Topics 20

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=20,
                                           alpha='auto',
                                           per_word_topics=True)

# Create Visualization (Save HTML)

The easiest way to create the visualization is to reveal it in the Google Colab notebook and save it as an html file that you can view on your browser. 

In [ ]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

In [ ]:
vis = gensimvis.prepare(lda_model, corpus, id2word)

#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')

In [ ]:
pyLDAvis.save_html(vis, '/content/LDAviz.html')

In [ ]:
pyLDAvis.display(vis)

# Topic Modeling Model - 60 Topics

In [ ]:
lda_model60 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=60,
                                           random_state=100,
                                           update_every=2,
                                           chunksize=100,
                                           passes=20,
                                           iterations=200,
                                           alpha='auto',
                                           per_word_topics=True)

# Create Visualization (Save HTML)

The easiest way to create the visualization is to reveal it in the Google Colab notebook and save it as an html file that you can view on your browser. 

In [ ]:
vis60 = pyLDAvis.gensim.prepare(lda_model60, corpus, id2word)
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')

In [ ]:
pyLDAvis.save_html(vis60, '/content/LDAviz60.html')

In [ ]:
pyLDAvis.display(vis60)

# Serve Visualization in Browser

You can also serve the visualization locally in the browser using the below chunk of code. Beware that caching in your browser and other issues, such as ad-blockers, may require some debugging to get this working on your machine. 

In [ ]:
#pyLDAvis.enable_notebook()
#pyLDAvis.show(vis)